# Logic Test Notebook
`run.py`의 실행 로직을 기반으로 특정 프로젝트의 메인 로직을 테스트합니다.

In [1]:
import os
import sys
import importlib
from pkg.utils.blackboard import initialize_global_blackboard, GlobalBlackboard
import time

# 1. 프로젝트 루트 경로 설정 (노트북 위치 기준)
project_root = os.path.abspath('')
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project Root: {project_root}")

# 2. 실행할 프로젝트 이름 설정
PROJECT_NAME = "shimadzu_logic"

# 3. Blackboard 초기화 (run.py와 동일한 경로 규칙 사용) 
blackboard_path = os.path.join(project_root, f"projects/{PROJECT_NAME}/configs/blackboard.json")

if os.path.exists(blackboard_path):
    initialize_global_blackboard(blackboard_path)
    print(f"✅ Blackboard initialized from: {blackboard_path}")
else:
    print(f"❌ Blackboard config not found at: {blackboard_path}")

# 4. 프로젝트 메인 모듈 로드 및 실행
module_name = f"projects.{PROJECT_NAME}.main"

try:
    # 소스 코드 수정 시 즉시 반영하기 위해 reload 처리
    if module_name in sys.modules:
        module = importlib.reload(sys.modules[module_name])
    else:
        module = importlib.import_module(module_name)

    if hasattr(module, "main"):
        print(f"🚀 Starting {module_name}.main()...\n")
        module.main()
        bb = GlobalBlackboard()
    else:
        print(f"⚠️ Error: Module '{module_name}' does not define main()")
except Exception as e:
    print(f"❌ Error running project: {e}")
    import traceback
    traceback.print_exc()

Project Root: c:\Users\S\Documents\GitHub\Shimadzu
Blackboard initialized from c:\Users\S\Documents\GitHub\Shimadzu\projects/shimadzu_logic/configs/blackboard.json
Blackboard initialized from c:\Users\S\Documents\GitHub\Shimadzu\projects/shimadzu_logic/configs/blackboard.json
✅ Blackboard initialized from: c:\Users\S\Documents\GitHub\Shimadzu\projects/shimadzu_logic/configs/blackboard.json
🚀 Starting projects.shimadzu_logic.main.main()...

[SYSTEM] Starting shimadzu_logic System...
[Indy7] Attempting to connect to robot at 192.168.2.20
[Indy7] Auto mode enabled.
[Indy7] Robot connection and initial setup successful.
[LOGIC] MQTT 통신 스레드를 시작합니다.
[LOGIC] MQTT loop_forever를 시작합니다. (Host: 127.0.0.1:1883)
[LOGIC] 브로커 연결 성공
[LOGIC] 토픽 구독 성공: /ui/cmd
[LOGIC] 토픽 구독 성공: /binpick/evt


2026-01-08 16:23:41,442 [INFO] Successfully connected to the database.


configs robot : 192.168.2.20
configs remot io : 192.168.2.40
configs mqtt : 127.0.0.1
configs shimadzu : 192.168.2.162 : 5000
실행 모드 : Run 모드
Initializing Device FSM...
[device] Debug mode : 0
[device] RemoteIO configs : 192.168.2.40
[device] Shimadzu configs : 192.168.2.162 : 5000
[device] Gauge Initialized
[device] remote I/O Initialized


2026-01-08 16:23:43,669 [ERROR] Generic message 'forward_open' failed: Service not supported
2026-01-08 16:23:43,671 [ERROR] forward_open failed - Service not supported


[device] QR Reader Initialized
[ShimadzuClient] Init Response: True
[ShimadzuClient] Connected to Shimadzu Device Successfully.
[device] Error in _thread_tower_lamp_controller: 'str' object has no attribute 'get'
[device] All device Init Complete
[device] _thread_comm_status_updater started
[device] Error in _thread_tower_lamp_controller: 'str' object has no attribute 'get'
[device] Error in _thread_tower_lamp_controller: 'str' object has no attribute 'get'
[device] SOL_SENSOR is ON. Performing initial device setup.
[device] lamp on
[device] Lamp On Command Sent Successfully.
[device] Aligner is already in pull state.
[device] Indicator stand is already down.
[device] enter ConnectingStrategy
[device] Attempting to connect to devices.
Device FSM initialized.
Initializing Robot FSM...
[Robot] Attempting to connect to robot controller...
Robot FSM initialized.
[Logic] Starting connection checks for all modules (Device & Robot).
Logic FSM initialized.
[Logic] All modules connected success

[device] ASK_SYS_STATUS timeout after 5.0s - No response from Shimadzu
[device] ASK_SYS_STATUS timeout after 5.0s - No response from Shimadzu
2026-01-08 16:25:45: [Robot FSM] Violation detected [indy_state=OP_VIOLATE, violation_code=512]
[Robot] exit RobotReadyStrategy with event: RobotEvent.VIOLATION_DETECT
Robot Violation Detected: HW_VIOLATION
[Robot] exit RobotErrorStrategy with event: RobotEvent.RECOVER
[Robot] Starting recovery process (SW/HW Reset).
[Robot] exit RobotRecoveringStrategy with event: RobotEvent.DONE
[Robot] Ready and waiting for commands.
2026-01-08 16:25:45: [Robot FSM] Violation detected [indy_state=OP_VIOLATE, violation_code=512]
[Robot] exit RobotReadyStrategy with event: RobotEvent.VIOLATION_DETECT
Robot Violation Detected: HW_VIOLATION
[Robot] exit RobotErrorStrategy with event: RobotEvent.RECOVER
[Robot] Starting recovery process (SW/HW Reset).
[Robot] exit RobotRecoveringStrategy with event: RobotEvent.DONE
[Robot] Ready and waiting for commands.
2026-01-08

In [2]:
robot_s = bb.get("robot/fsm/strategy")
device_s = bb.get("device/fsm/strategy")
logic_s = bb.get("logic/fsm/strategy")
auto_cmd = bb.get("ui/cmd/auto/tensile")

print(f"robot {robot_s}\ndevice {device_s}\nlogic {logic_s}\nauto cmd {auto_cmd}")

robot {'state': 'READY', 'strategy': 'RobotReadyStrategy'}
device {'state': 'READY', 'strategy': 'ReadyStrategy'}
logic {'state': 'IDLE', 'strategy': 'LogicIdleStrategy'}
auto cmd 0


In [5]:
# # 수동 명령 실행
#             if manual_smz_cmd == 1:
#                 # START_MEASUREMENT - lotname 필요
#                 lot_name = bb.get("process_status/lot_name", "")
#                 context.smz_start_measurement(lot_name)
#             elif manual_smz_cmd == 2:
#                 context.smz_stop_measurement()
#             elif manual_smz_cmd == 3:
#                 context.smz_ask_sys_status()
#             elif manual_smz_cmd == 4:
#                 context.smz_are_you_there()
#             elif manual_smz_cmd == 5:
#                 context.smz_ask_preload()
#             elif manual_smz_cmd == 6:
#                 # ASK_REGISTER - regist_data 필요
#                 qr_no = bb.get("process_status/qr_no", "")
#                 batch_id = bb.get("process_status/batch_id", "")
#                 regist_data = {
#                     "qr_no": qr_no,
#                     "batch_id": batch_id
#                 }
#                 context.smz_ask_register(regist_data)

bb.set("manual/device/shimadzu",1) 

In [ ]:
bb.get("device/shimadzu/run_state")

{'command': 'SYS_STATUS',
 'params': {'MODE': 'A', 'RUN': 'N', 'LOAD': '000.0000', 'TEMP': '025.0'}}

In [ ]:
logic = bb.get("logic/fsm/strategy")
device_fsm_state = bb.get("device/fsm/strategy")
robot_fsm_state = bb.get("robot/fsm/strategy")
print(f"logic: {logic}, device: {device_fsm_state}, robot: {robot_fsm_state}")

In [ ]:
# 인디게이터 두께 측정
bb.set("manual/device/tester",9)
time.sleep(2)
bb.get("device/gauge/thickness")

In [ ]:
# 두께 측정기 받침 올리기
bb.set("manual/device/tester",12)

In [ ]:
# 두께 측정기 받침 내리기
bb.set("manual/device/tester",13)

In [ ]:
# 시편 정렬기 풀기
bb.set("manual/device/tester",7)

In [ ]:
# 시편 정렬기 시편 정렬
bb.set("manual/device/tester",6)

In [ ]:
bb.get("device/remote/input/SOL_SENSOR")

In [ ]:
bb.get("process_status")

In [ ]:
# setting화면 보낸 데이터
bb.get("system/setting/context")

In [ ]:
data = {
    "batch_id" : "BT-20251226",
    "status" : "시험중"
}
bb.set("process_status/batch_info",data)

In [ ]:
bb.get("process_status/batch_info")

In [ ]:
runtime = {
    "starttime" : "16:39:00",
    "elapsedtime" : "05:13:12"
}
bb.set("process_status/runtime",runtime)

In [ ]:
bb.get("process_status/runtime")

In [ ]:
robot_s = bb.get("robot/fsm/strategy")
device_s = bb.get("device/fsm/strategy")
logic_s = bb.get("logic/fsm/strategy")

print(f"robot {robot_s}\ndevice {device_s}\nlogic {logic_s}")